# Fewshot Retrieval Classifier in Langchain and Argilla

In this notebook we will show how to use the Langchain and Argilla libraries to build a fewshot retrieval classifier. The classifier will be able to classify a given text into a class by retrieving the most similar text from a set of examples. We will argilla to review the classifications.

In [13]:
!pip install -qqq langchain \
                  langchain-chroma \
                  langchain-huggingface \
                  datasets

We're going to use a basic prompt with a question and topic.

In [66]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("Question: {question}\n Topic of the question: {topic}")

We will load a toy dataset with examples of yahoo answers and questions and topics of the question. 

In [ ]:
from datasets import load_dataset

ds = load_dataset(("burtenshaw/yahoo_answers_topics"))
topics = list(map(lambda s: s.lower(), set(ds["train"]["topic"])))

This is what the prompt to the LLM will look like:

In [67]:
print(example_prompt.invoke(input=ds["train"][0]).to_string())

Question: why doesn't an optical mouse work on a glass table?
 Topic of the question: Computers & Internet


## Define a Fewshot Prompt template in Langchain

We will define a prompt template in Langchain that will be used to classify the text. The template will be a question and a topic. The question will be the text to classify and the topic will be the class to classify

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=ds["train"].to_list(),
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

Now, we can print our prompt from the template with examples. You can see that the language model will be given answers to previous questions and topics.

In [27]:

new_question = "Why doesn't my mouse work on my glass table?"

print(prompt.invoke({"input": new_question}).to_string())

Question: why doesn't an optical mouse work on a glass table?
Computers & Internet

Question: What is the best off-road motorcycle trail ?
Sports

Question: What is Trans Fat? How to reduce that?
Health

Question: How many planes Fedex has?
Business & Finance

Question: In the san francisco bay area, does it make sense to rent or buy ?
Business & Finance

Question: What's the best way to clean a keyboard?
Computers & Internet

Question: Why do people blush when they are embarrassed?
Science & Mathematics

Question: Is Lin Qingxia (aka Brigitte Lin) "the most beautiful woman in Chinese cinema?"
Entertainment & Music

Question: What is the origin of "foobar"?
Computers & Internet

Question: How the human species evolved?
Science & Mathematics

Question: Who said the statement below and what does it mean?
Education & Reference

Question: How do I find an out of print book?
Education & Reference

Question: What are some tips on finding a good mortgage broker?
Business & Finance

Question: 

# Retrieving similar Fewshot examples

We don't just want to throw any examples in to our prompt. Ideally, we will use few shot examples that are similar to the question we're trying to classify. 

To do that, we will use a vector index, embeddings, and cosine similarity to retrieve the most similar examples to the question we're trying to classify. We can implement this with three langchain integrations:

- `Chroma`: A vector index to store the embeddings of the examples
- `HuggingFaceEndpointEmbeddings`: To get the embeddings of the question, served on a Hugging Face endpoint
- `SemanticSimilarityExampleSelector`: To retrieve the most similar examples to the question from the database

In [68]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_chroma import Chroma

hf = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-mpnet-base-v2",
    task="feature-extraction",
    huggingfacehub_api_token="<your-huggingfacehub-api-token>",
)

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=ds["train"].to_list(),
    embeddings=hf,
    vectorstore_cls=Chroma,
    k=5,
)

Now we can use out example selector to retrieve the most similar examples to the question we're trying to classify.

In [69]:
selected_examples = example_selector.select_examples(
    input_variables={"question": new_question}
)

print(*selected_examples, sep="\n")

{'answer': 'Optical mice use an LED and a camera to rapidly capture images of the surface beneath the mouse.  The infomation from the camera is analyzed by a DSP (Digital Signal Processor) and used to detect imperfections in the underlying surface and determine motion. Some materials, such as glass, mirrors or other very shiny, uniform surfaces interfere with the ability of the DSP to accurately analyze the surface beneath the mouse.  \\nSince glass is transparent and very uniform, the mouse is unable to pick up enough imperfections in the underlying surface to determine motion.  Mirrored surfaces are also a problem, since they constantly reflect back the same image, causing the DSP not to recognize motion properly. When the system is unable to see surface changes associated with movement, the mouse will not work properly.', 'question': "why doesn't an optical mouse work on a glass table?", 'topic': 'Computers & Internet'}
{'answer': 'Optical mice use an LED and a camera to rapidly cap

# Update the prompt with the retrieved examples

We can now update our FewShot prompt template with the retrieved examples based on the question. This will give the language model more context to classify the question.

In [71]:
prompt = FewShotPromptTemplate(
    examples=selected_examples,
    example_prompt=example_prompt,
    suffix="Question: {input}\nTopic of the question:",
    input_variables=["input"],
)
print(prompt.invoke({"input": new_question}).to_string())

Question: why doesn't an optical mouse work on a glass table?
 Topic of the question: Computers & Internet

Question: why doesn't an optical mouse work on a glass table?
 Topic of the question: Computers & Internet

Question: why doesn't an optical mouse work on a glass table?
 Topic of the question: Computers & Internet

Question: What's the best way to clean a keyboard?
 Topic of the question: Computers & Internet

Question: What's the best way to clean a keyboard?
 Topic of the question: Computers & Internet

Question: Why doesn't my mouse work on my glass table?
Topic of the question:


# Define a language model in Langchain

We will use another Hugging Face endpoint to serve the language model. This time we will use Llama-3.1-8B-Instruct by meta.

In [72]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = ChatHuggingFace(
    llm=HuggingFaceEndpoint(
        repo_id="meta-llama/Llama-3.1-8B-Instruct",
        task="text-generation",
        max_new_tokens=20,
        do_sample=False,
        repetition_penalty=1.03,
    ),
    verbose=True,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/ben/.cache/huggingface/token
Login successful


# Define the Fewshot Retrieval Classifier in Langchain

Now we can define the Fewshot Retrieval Classifier in Langchain. The classifier will use the prompt template, the example selector, and the language model to classify the question. The model we're using is a chat model, so we will need to represent the question as a chat message.

In [77]:
def call_llama(question):
    selected_examples = example_selector.select_examples(
        input_variables={"question": question}
    )
    prompt = FewShotPromptTemplate(
        prefix=f"Select the topic of the question from these topics : {topics}\n Return only the topic of the question\n",
        examples=selected_examples,
        example_prompt=example_prompt,
        suffix="Question: {input}\nTopic:",
        input_variables=["input"],
    )
    messages = [{"role": "user", "content": prompt.format(input=question)}]
    return llm.invoke(input=messages).content


call_llama(new_question)

'Computers & Internet'

Great. That works on one example. Now let's try it on a few examples.

In [62]:
predictions = []

for sample in ds["test"]:
    sample["topic"] = call_llama(sample["question"]).lower()
    predictions.append(sample)

print(*predictions[:5], sep="\n")

{'topic': 'computers & internet', 'question': 'What makes friendship click?', 'answer': "good communication is what does it.  Can you move beyond small talk and say what's really on your mind.  If you start doing this, my expereince is that potentially good friends will respond or shun you.  Then you know who the really good friends are."}
{'topic': 'computers & internet', 'question': 'Why does Zebras have stripes?', 'answer': 'this provides camouflage - predator vision is such that it is usually difficult for them to see complex patterns'}
{'topic': 'computers & internet', 'question': 'What did the itsy bitsy sipder climb up?', 'answer': 'waterspout'}
{'topic': 'computers & internet', 'question': 'What is the difference between a Bachelors and a Masters degree?', 'answer': "One difference between a Bachelors and a Masters degree are the requirements. To gain eligibility for a Bachelors, you must have at least a GED and have scored well enough on the SAT. To be eligible for a Masters D

# Argilla

Now we can use Argilla to review the classifications and see whether the retrievals are helping the model to classify the questions.

In [78]:
from uuid import uuid4
import argilla as rg

client = rg.Argilla()

dataset = rg.Dataset(
    name=f"langchain-fewshot-classification-uuid-{uuid4()}",
    settings=rg.Settings(
        fields=[
            rg.TextField(name="question"),
            rg.TextQuestion(name="answer"),
        ],
        questions=[rg.LabelQuestion(name="topic", labels=topics)],
    ),
)

dataset.create()

dataset.records.log(predictions)

/Users/ben/code/argilla-cookbook/.venv/lib/python3.12/site-packages/argilla/datasets/_resource.py:250: UserWarning: Workspace not provided. Using default workspace: argilla id: 31fb2d4c-46f8-4698-8dcb-27555e78c092
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")


Dataset(id=UUID('f1ce36a4-ee82-4500-b884-28abc143f2d5') inserted_at=datetime.datetime(2024, 10, 3, 12, 31, 52, 25541) updated_at=datetime.datetime(2024, 10, 3, 12, 31, 53, 28625) name='langchain-fewshot-classification-uuid-416fc3c1-ac03-477e-92af-a90bc8722688' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('31fb2d4c-46f8-4698-8dcb-27555e78c092') last_activity_at=datetime.datetime(2024, 10, 3, 12, 31, 53, 28625))